In [ ]:
import pandas as pd
import glob
from  geopy.geocoders import Nominatim
import requests
import json
import qwikidata
import qwikidata.sparql


from requests import get
from bs4 import BeautifulSoup
import re
from IPython.display import display_html

import numpy as np


In [ ]:
#os.chdir('/Users/karlijnrozestraten/Documents/Development.nosync')

#all_files = os.listdir('Woonplaatsen_NL')

#li = []

#for filename in all_files:
  #  df = pd.read_csv('Woonplaatsen_NL/' + filename,sep=';', header=1)
   # df.drop(df.tail(1).index,inplace=True)
    #li.append(df)

#city_names = pd.concat(li, axis=0, ignore_index=True)



#city_names.to_csv('city_names.csv')


## Load CBS city names

In [ ]:
# from: https://opendata.cbs.nl/#/CBS/nl/dataset/84734NED/table
# select alle places
cities = pd.read_csv('Woonplaatsen_NL_complete.csv', sep=';')
cities = cities.set_axis(['Woonplaatsen','Gemeente', 'Provincie','Landsdeel'], axis=1, inplace=False)
cities['Provincie'] = cities['Provincie'].str.replace(' ','')
cities.to_csv('cities.csv')

## Retrieve information from Wikipedia, as much as possible

In [ ]:
no_places = ['Abbekerk','Basse', 'Bruinehaar', 'Barsingerhorn','Bloemendaal', 'Collendoorn', 'Groet','Driehuizen','Delden','Driewegen', 'Hem', 'Holten', 'Hoek', 'Lutten', 'Markelo', 'Ressen', 'Morra','Nieuwerkerk','Oostburg','Halfweg','Riel','Houwerzijl','Onstwedde', 'Welsum','Julianadorp','Koedijk','Noordeinde','Purmer',
            'Schiphol','Schiphol-Rijk','Spaarndam','Wieringerwaard','Zwaag','Aalst','Doesburg','Ewijk','Hattem','Hattemerbroek',
            'Heelweg','Herveld','Klarenbeek','Lienden','Ommeren','Ooij','Poederoijen','Steenenkamer','Vethuizen','Voorst','Voorthuizen',
            'Wapenveld','Well','Westendorp','Wijnbergen','Zelhem','Abbega','Baard','Bornwird','Dokkum','Ee','Exmorra','Hallum',
            'Hantum','Hemrik','Indijk','Midsland','Miedum','Oosthem','Siegerswoude','Sumar','Surhuizum','Teerns','Barendrecht',
            'Katwijk','Stolwijk','Strijen','Strijensas','Zevenhuizen','Alphen','Baarle-Nassau','Bakel','Chaam','Doeveren',
            'Gastel','Heeze','Herpt','Hoeven','Nispen','Noordhoek','Oisterwijk','Oosteind','Ulvenhout','Wernhout','Wouw','Zevenbergen',
            'Aduard','Borgercompagnie','Finsterwolde','Jonkersvaart','Nieuwolda','Noordhorn','Oldenzijl','Oostwold','Oudezijl','Startenhuizen',
            'Vierhuizen','Vlagtwedde','Wildervank']               


In [ ]:
# create dataframe for output
results = pd.DataFrame(columns = ['Place','Link','Province','Coordinates', 'Population', 'Lattitude', 'Longitude'])


# retrieve information from wikipedia page about provinces
response = get('https://nl.wikipedia.org/wiki/Categorie:Lijsten_van_Nederlandse_plaatsen_naar_provincie')
html_soup = BeautifulSoup(response.text, 'html.parser')
data = html_soup.find('div',{'class':'mw-category'})

# retrieve the correct links for the pages with all places per province
links = data.findAll('a')
provinces_links = []
for link in links:
    if link['href'][6:22] == 'Lijst_van_steden':
        plink = 'https://nl.wikipedia.org'+link['href']
        provinces_links.append(plink)
# create a list of all the unique provinces to make iteration possible
provinces = cities['Provincie'].unique()

# start the iteration for each provinces
for province in provinces:
    item_list = []

    # retrieve the overview page of the province
    pl = re.compile('.*\D({})$'.format(province))
    for find_link in provinces_links:
        m = pl.match(find_link)

        if m:
            province_page = get(find_link)
            html_soup = BeautifulSoup(province_page.text, 'html.parser')        
            # expend the iteration by looping for each place in a province. Create a list of the places in a province
            places_in_province = cities.loc[cities['Provincie'] == province, 'Woonplaatsen']
            for place in places_in_province:
                if place not in no_places:

                    # The format of the province Drenthe is slightly different then the other provinces. The approach differs slightly
                    if province == 'Drenthe':
                        # Only retrieve the elements where the information we need is in
                        page_data = html_soup.findAll('table',{'class':''})
                        tr_list = [tr.findAll('tr') for tr in page_data]
                        place_link_list = [li.find('a') for item in tr_list for li in item if li.find('a')!= None]
                        # Get the link for the place page in order to retrieve information about the place

                        for place_link in place_link_list:
                            if place_link != None:
                                if place_link['href'][6:(len(place)+6)] == place:

                                    placelink = 'https://nl.wikipedia.org'+place_link['href']

                                    # Get access to the infobox on the pace
                                    place_page = get(placelink)
                                    html_soup = BeautifulSoup(place_page.text, 'html.parser')
                                    place_data = html_soup.find('table',{'class':'infobox'})
                                    # Retrieve the coordinate and population information
                                    table_df = pd.read_html(str(place_data))[0]
                                    co = table_df.loc[table_df[0] == 'Coördinaten',[1]].values[0]
                                    pop = table_df[table_df[0].str.match(r'Inwoners.*')==True][1].values[0]

                                    # Transform the coordinates to decimals
                                    split = str(co).rsplit(',')
                                    lat = split[0]
                                    long = split[1]
                                    lat_ = re.findall('\d+', lat)
                                    long_ = re.findall('\d+',long)
                                    lat_ = [int(c) for c in lat_]
                                    long_ = [int(c) for c in long_]

                                    if len(lat_) != 3:
                                        lat_.append(0)
                                    latdd = lat_[0] + (lat_[1]/60) + (lat_[2]/3600)

                                    if len(long_) !=3:
                                        long_.append(0)
                                    longdd = long_[0] + (long_[1]/60) + (long_[2]/3600)
                                    # Add the information to the dataframe
                                    results.loc[len(results)] = [place, placelink, province, co, pop, latdd, longdd]
                                           

                    else:
                        # Retrieve information form the province overview page
                        page_data = html_soup.find('div',{'class':'mw-parser-output'})
                        tr_list = page_data.findAll('tr')
                        # Create a list with the places in the province and the link to it
                        for item in tr_list:
                            item_list.append(item.findAll('li'))   
                        places_list = [li.find('a') for li_list in item_list for li in li_list if li != None]
                        # Get the link for the place page in order to retrieve information about the place
                        for place_link in places_list:
                            if place_link != None and place_link['href'][6:] != 'Baarlo_(Zwartewaterland)':

                                if place_link['href'][6:(len(place)+6)] == place:
                                    placelink = 'https://nl.wikipedia.org'+place_link['href']

                                    # Get access to the infobox on the pace
                                    place_page = get(placelink)
                                    html_soup = BeautifulSoup(place_page.text, 'html.parser')
                                    place_data = html_soup.find('table',{'class':'infobox'})
                                    # Retrieve the coordinate and population information
                                    table_df = pd.read_html(str(place_data))[0]
                                    co = table_df.loc[table_df[0] == 'Coördinaten',[1]].values[0]
                                    pop = table_df[table_df[0].str.match(r'Inwoners.*')==True][1].values[0]

                                    # Transform the coordinates to decimals
                                    split = str(co).rsplit(',')
                                    lat = split[0]
                                    long = split[1]
                                    lat_ = re.findall('\d+', lat)
                                    long_ = re.findall('\d+',long)
                                    lat_ = [int(c) for c in lat_]
                                    long_ = [int(c) for c in long_]

                                    if len(lat_) != 3:
                                        lat_.append(0)
                                    latdd = lat_[0] + (lat_[1]/60) + (lat_[2]/3600)

                                    if len(long_) !=3:
                                        long_.append(0)
                                    longdd = long_[0] + (long_[1]/60) + (long_[2]/3600)
                                    # Add the information to the dataframe
                                    results.loc[len(results)] = [place, placelink, province, co, pop, latdd, longdd]
                                    

results.to_csv('results.csv')
print(results)




## After manual adjustments, read in complete dataset

In [ ]:
cities_complete = pd.read_csv('cities_edit.csv', sep=';')
# Drop the columns we don't need
cities_complete = cities_complete.drop(columns = ['Unnamed: 0','L1','l2','L3','Lat','L1.1','L1.2','L.1.3','Long','Unnamed: 17'])
# Drop places without population data
cities_complete = cities_complete[cities_complete['Population'] != 'None']
# Change values op column population into integers
cities_complete['Population'] = cities_complete['Population'].astype(int)

cities_complete.head()

In [ ]:
# Select the rows were Lattitude and Longitude are empty
missinglong_lat = cities_complete.loc[(cities_complete['Lattitude'].isnull()) & (cities_complete['Longitude'].isnull())]

# For each coordinate in the new dataframe we will calculate the Longitude and Lattitude
for coordinates in missinglong_lat['Coordinates']:
    
    # Get the real index of the row in the big df
    index_co = missinglong_lat[missinglong_lat['Coordinates']== coordinates].index.values
    place = missinglong_lat['Woonplaatsen'][index_co].values
    real_index = cities_complete.loc[(cities_complete['Coordinates'] == coordinates) & (cities_complete.Woonplaatsen == place[0])].index.values
    
    # Calculate the coorinates to longitude and lattitude decimals
    split = str(coordinates).rsplit(',')
    lat = split[0]
    long = split[1]
    lat_ = re.findall('\d+', lat)
    long_ = re.findall('\d+',long)
    lat_ = [int(c) for c in lat_]
    long_ = [int(c) for c in long_]
    
    if len(lat_) == 2:
        lat_.append(0)
    if len(lat_) == 1:
        lat_ = lat_ + [0,0]
    latdd = lat_[0] + (lat_[1]/60) + (lat_[2]/3600)

    if len(long_) == 2:
        long_.append(0)
    if len(long_) == 1:
        long_ = long_ + [0,0]
    longdd = long_[0] + (long_[1]/60) + (long_[2]/3600)
    
    # Put the longitude and lattitude data into the big dataframe
    cities_complete.at[index_co,'Lattitude'] = latdd
    cities_complete.at[index_co,'Longitude'] = longdd

# Write the complete dataframe to CSV.
cities_complete.to_csv('cities_complete_clean.csv')

    


In [ ]:
# Make a selection of cities where the population is equal or higher than 20000 and write that to a new csv.
cities_20k = cities_complete.loc[(cities_complete['Population']>=20000)]
cities_20k.to_csv('cities_20k.csv')
cities_20k